In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as pl
from scipy.special import legendre



def train_model(width):
    # Set seed for reproducibility
    torch.manual_seed(42)

    # === Configurable parameters ===
    NUM_LAYERS = 1           # Number of hidden layers
    NEURONS_PER_LAYER = width   # Neurons per hidden layer
    ENSEMBLE_SIZE = 10       # Number of networks in ensemble
    NUM_EPOCHS = 1000
    LEARNING_RATE = 0.1

    # === Define a simple dataset ===
    def true_function(x):
        P = legendre(4)  # 4th-degree Legendre polynomial, change as needed
        x_np = x.numpy().squeeze()  # Convert to NumPy for scipy
        y_np = P(x_np)              # Evaluate polynomial
        return torch.tensor(y_np, dtype=torch.float32).unsqueeze(1)


    x_train = torch.linspace(-4, 4, 20).unsqueeze(1)  # shape: (100, 1)
    y_train = true_function(x_train)

    # === Define a customizable feedforward network ===
    class SimpleNet(nn.Module):
        def __init__(self, num_layers, neurons_per_layer):
            super(SimpleNet, self).__init__()
            layers = []

            input_dim = 1
            for _ in range(num_layers):
                layer = nn.Linear(input_dim, neurons_per_layer)
                nn.init.normal_(layer.weight, mean=0.0, std=1.0)  # Gaussian weight init
                nn.init.normal_(layer.bias, mean=0.0, std=1.0)    # Gaussian bias init
                layers.append(layer)
                layers.append(nn.Tanh())  # activation
                input_dim = neurons_per_layer

            # Output layer
            final_layer = nn.Linear(input_dim, 1)
            nn.init.normal_(final_layer.weight, mean=0.0, std=1.0)
            nn.init.normal_(final_layer.bias, mean=0.0, std=1.0)
            layers.append(final_layer)

            self.net = nn.Sequential(*layers)

        def forward(self, x):
            return self.net(x)

    # === Train a single network ===
    losses = []

    def train_network(model, x_train, y_train):
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        for epoch in range(NUM_EPOCHS):
            optimizer.zero_grad()
            output = model(x_train)
            loss = criterion(output, y_train)
            loss.backward()
            optimizer.step()

        losses.append(loss.item())

        return model

    # === Train ensemble of networks ===
    ensemble_outputs = []

    for i in range(ENSEMBLE_SIZE):
        model = SimpleNet(NUM_LAYERS, NEURONS_PER_LAYER)
        trained_model = train_network(model, x_train, y_train)
        with torch.no_grad():
            output = trained_model(x_train)
            ensemble_outputs.append(output.numpy())


    ensemble_outputs = np.stack(ensemble_outputs, axis=0)  # shape: (ensemble, samples, 1)

    # === Compute statistics across ensemble ===
    mean_output = np.mean(ensemble_outputs, axis=0).squeeze()
    std_output = np.std(ensemble_outputs, axis=0).squeeze()

    # === Plot ===
    x_np = x_train.numpy().squeeze()
    y_np = y_train.numpy().squeeze()


    loss = np.mean(losses)
    return loss

loss = []
neurons = []
for i in range (1, 50):
    loss.append(train_model(i))
    neurons.append(1/i)
pl.plot(neurons, loss)
pl.show()
